In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

# Initialize Spark session with Hive support
spark = SparkSession.builder \
    .appName("Example") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

24/06/03 14:50:59 WARN Utils: Your hostname, Arbnors-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.11.6.58 instead (on interface en0)
24/06/03 14:50:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 14:51:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/03 14:51:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark.sql("SELECT 1+1").show(1)

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



In [3]:
data = spark.read.json("data/flight-data/json/2010-summary.json").createOrReplaceTempView("sqlData")

In [4]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) 
FROM sqlData 
GROUP BY DEST_COUNTRY_NAME 
HAVING sum(count) > 30""")\
.where("DEST_COUNTRY_NAME LIKE 'S%'").show(5, False)

+---------------------+----------+
|DEST_COUNTRY_NAME    |sum(count)|
+---------------------+----------+
|Sweden               |65        |
|Spain                |422       |
|Saint Kitts and Nevis|113       |
|South Korea          |683       |
|Sint Maarten         |61        |
+---------------------+----------+
only showing top 5 rows



In [5]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) 
FROM sqlData 
GROUP BY DEST_COUNTRY_NAME 
HAVING len(DEST_COUNTRY_NAME) < 5""").show(5, False)

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|Fiji             |53        |
|Peru             |212       |
|Cuba             |243       |
+-----------------+----------+



In [9]:
# Create new table from the existing one

spark.sql("""

CREATE TABLE flights (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path '/Users/arbnormorina/Desktop/Spark/Apache-Spark-with-DataBricks/data/flight-data/json/2010-summary.json')

""")

24/06/03 14:55:56 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider JSON. Persisting data source table `spark_catalog`.`default`.`flights` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [10]:
spark.sql("SELECT * FROM flights").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



## External Tables or unmanaged table

Spark will manage the table’s metadata; however, the files are not managed by Spark at all. 

The data will be read but it will not be stored the data are not owned.

In [13]:
spark.sql("""

CREATE EXTERNAL TABLE flightsExternal(
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION '/Users/arbnormorina/Desktop/Spark/Apache-Spark-with-DataBricks/data/flight-data-hive'
""")

DataFrame[]

In [14]:
spark.sql("SELECT * FROM flightsExternal").show(10, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova          |United States      |1    |
+-----------------+-------------------+-----+
only showing top 10 rows



In [15]:
spark.sql("DESCRIBE TABLE flightsExternal").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   NULL|
|ORIGIN_COUNTRY_NAME|   string|   NULL|
|              count|   bigint|   NULL|
+-------------------+---------+-------+

